In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

# Create a dataframe from JSON file and print its schema

In [5]:
df = spark.read.format("json").load("/home/supriya/Documents/Supriya/UMBC_IS/UMBC_Courses/IS_700_IndStudy/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")

In [6]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]>

# Another way of displaying schema and see its StructType

In [7]:
spark.read.format("json").load("/home/supriya/Documents/Supriya/UMBC_IS/UMBC_Courses/IS_700_IndStudy/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

# Accessing dataframe's columns

In [8]:
spark.read.format("json").load("/home/supriya/Documents/Supriya/UMBC_IS/UMBC_Courses/IS_700_IndStudy/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

# Creating temporary view from dataframe to run SQL queries

In [9]:
 df.createOrReplaceTempView("dfTable")

# Creating a dataframe manually

In [10]:
 from pyspark.sql import Row
 from pyspark.sql.types import StructField, StructType, StringType, LongType
 manualSchema = StructType([StructField("some",StringType(),True), StructField("col",StringType(),True),StructField("names", LongType(),False)])
 row1 = Row("Hello",None,1)
 manualDF= spark.createDataFrame([row1],manualSchema)
 manualDF.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



# Select operation on dataframe - different ways

In [11]:
 df.select("Dest_country_name").show(2)

+-----------------+
|Dest_country_name|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [12]:
 df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [13]:
 from pyspark.sql.functions import expr, col, column
 df.select(expr("DEST_COUNTRY_NAME"),col("DEST_COUNTRY_NAME"),column("DEST_COUNTRY_NAME")).show(2)


+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [14]:
 df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [15]:
 df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [16]:
 df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [17]:
 df.selectExpr("*","(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) AS withinCountry").show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [18]:
 df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



# Using literals as columns - literals are constant values

In [19]:
 from pyspark.sql.functions import lit
 df.select(expr("*"), lit(1).alias("one")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|one|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



# Adding new column to a dataframe - using a literal, using an expression to store boolean, using an existing column

In [20]:
 df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [21]:
 df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [22]:
 df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

# Renaming a column

In [23]:
 df.withColumnRenamed("DEST_COUNTRY_NAME", "DEST").columns

['DEST', 'ORIGIN_COUNTRY_NAME', 'count']

# Using Escape character "`" backtick character to skip reserved characters such as space or dash.

In [25]:
 dfLongName = df.withColumn("This Long Column-Name", expr("ORIGIN_COUNTRY_NAME"))

In [26]:
 dfLongName.selectExpr("`This Long Column-Name`","`This Long Column-Name` as `new col`").show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



# Removing or Dropping column

In [27]:
dfLongName.drop("count").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'This Long Column-Name']

In [28]:
dfLongName.drop("ORIGIN_COUNTRY_NAME","This Long Column-Name").columns

['DEST_COUNTRY_NAME', 'count']

# Changing a column's Type by casting

In [29]:
 df.withColumn("count2", col("count").cast("long"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: bigint]

# Filtering Rows using single expression

In [30]:
 df.filter(col("count")<2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [31]:
 df.where("count<2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



# Filtering rows using multiple expressions - Spark executes all the filter expression at the same time irrespective of the given order

In [32]:
 df.where(col("count")<2).where(col("ORIGIN_COUNTRY_NAME")!="Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



# Getting unique rows

In [33]:
 df.select("ORIGIN_COUNTRY_NAME","DEST_COUNTRY_NAME").distinct().count()

256

In [34]:
 df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

# Getting random sample of a dataframe

In [36]:
 withReplacement = False
 fraction = 0.5
 seed = 5
 df.sample(withReplacement, fraction,seed).count()

126

# Random split - used in Machine learning algorithms to split data into validation data and test data

In [38]:
 dataSplit = df.randomSplit([0.25, 0.75],seed)
 dataSplit[0].count() > dataSplit[1].count()

False

# Concatinating 2 dataframes

In [40]:
 from pyspark.sql import Row
 schema = df.schema
 newRows = [Row("New Country","Other Country",5),Row("New Country 2","Other Country 2",1)]
 parallelizedRows = spark.sparkContext.parallelize(newRows)
 newDF = spark.createDataFrame(parallelizedRows, schema)
 df.union(newDF).where("count =1").where(col("ORIGIN_COUNTRY_NAME")!= "United States").show()


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 2|    1|
+-----------------+-------------------+-----+



# Sorting rows

In [41]:
 df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [42]:
 df.orderBy("count","DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [43]:
 df.orderBy(col("count"),col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [45]:
 from pyspark.sql.functions import desc, asc
 df.orderBy(expr("count desc")).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [46]:
 df.orderBy(col("count").desc(),col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows

